In [1]:
import sys
from pathlib import Path
import pickle
import pandas as pd
import mdtraj as md
import networkx as nx
import logging
from collections import namedtuple
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures as cf
import glob
import csv
from collections import defaultdict
import pandas as pd
import subprocess as sp
import shutil as sh
import MDAnalysis as mda
from itertools import combinations_with_replacement
from MDAnalysis.core.groups import AtomGroup
from MDAnalysis.analysis import distances

from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
from Bio.SeqUtils import seq1, seq3
sys.path.insert(0, "/home/pbarletta/freesasa-python/build/lib.linux-x86_64-3.10")
import freesasa

Chains = namedtuple('Chains', ['antibody', 'antigen'])
InterfaceAtoms = namedtuple('InterfaceAtoms', ['antibody', 'antigen'])
Atom = namedtuple('Atom', ['index', 'serial', 'element', 'is_sidechain', 'resSeq',
                  'resSeq_str', 'resname', 'chain_ID', 'chain_type', 'CDR'])

from abag_interactions_hydrophobic import *
from interactions_hydrophobic import *
           
data_dir = Path("/home/pbarletta/labo/22/migue/data")
expdata_dir = Path("/home/pbarletta/labo/22/migue/data/AB-Bind-Database-master")
hydro_dir = Path("/home/pbarletta/labo/22/migue/rtdos/hydro")
aux_dir = Path("/home/pbarletta/labo/22/migue/aux")
pdbs_dir = Path("/home/pbarletta/labo/22/migue/run/pdbs")
mutpdbs_dir = Path("/home/pbarletta/labo/22/migue/run/mut_pdbs")
bin_dir = Path("/home/pbarletta/labo/22/locuaz/bin")
evo_bin = Path(bin_dir, "evoef2/EvoEF2")

AA_LIST = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS",
           "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]
AA_LIST = ("D", "E", "S", "T", "R", "N", "Q", "H", "K", "A", "G", "I", 
    "M", "L", "V", "P", "F", "W", "Y", "C" )

/home/pbarletta/anaconda3/envs/mdtraj/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
expdata_df = pd.read_csv(Path(expdata_dir, "AB-Bind_experimental_data.csv"), encoding='latin-1')
pdb_list = tuple(sorted(set(expdata_df['#PDB'])))

abag_chains = {}
for partners, pdb_id in zip(expdata_df["Partners(A_B)"], expdata_df["#PDB"]):
    target, binder = partners.split("_")
    abag_chains[pdb_id] = Chains(antibody=binder, antigen=target) 

mut_list = []
with open(Path(mutpdbs_dir, "mut_list.txt"), "r") as f:
    for mute in f:
        mut_list.append(mute.strip())

pdbs_mut = defaultdict(list)
for mut in mut_list:
    this_pdb = mut.split('-')[0]
    pdbs_mut[this_pdb ].append(mut)

## hydrophobic

In [12]:
pdb_id = pdb_list[25]

ag_chains = abag_chains[pdb_id].antigen
ab_chains = abag_chains[pdb_id].antibody

In [14]:
pdb_filename = Path(pdbs_dir, pdb_id + ".pdb")

G = construct_graph(pdb_filename, ag_chains, ab_chains, CUTOFF_CARBONS)
pre_clusteres = get_putative_clusters(G)
clusters = merge_clusters(pre_clusteres, CUTOFF_CLUSTERS)
my_clusters = MDAtom_to_TAtom(pdb_filename, clusters)
draw_clusters(pdb_filename, ag_chains=ag_chains, clusters=my_clusters,
    filename=Path(aux_dir, "cclusters.py"))

In [18]:
pdbs_mut[pdb_id][19]

'3NPS-A:R48A'

In [17]:
mut_id = pdbs_mut[pdb_id][19]
pdb_filename = Path(mutpdbs_dir, mut_id + ".pdb")

G = construct_graph(pdb_filename, ag_chains, ab_chains, CUTOFF_CARBONS)
pre_clusteres = get_putative_clusters(G)
clusters = merge_clusters(pre_clusteres, CUTOFF_CLUSTERS)
my_clusters = MDAtom_to_TAtom(pdb_filename, clusters)
draw_clusters(pdb_filename, ag_chains=ag_chains, clusters=my_clusters,
    filename=Path(aux_dir, "1_clusters.py"))

In [11]:
with open(Path.joinpath(hydro_dir, "mut_hydrophobic.pkl"), "rb") as file:
        mut_hydrophobic_clusters = pickle.load(file)

-----